### Preparation 

In [2]:
import json
with open('secret_tokens.json') as f:
    tokens_dict = json.load(f)

### CodeLLama

In [ ]:
import importlib
importlib.reload(clw)

In [ ]:
import codellama_wrapper as clw
codellama_trial = clw.CodellamaEval(model_name = "codellama/CodeLlama-7b-hf", gpu_id=1)
codellama_trial.generate()

In [9]:
file_name = codellama_trial.model_process()

In [ ]:
codellama_trial.evaluate()

In [3]:
# If we have a json with the data, we can run just the eval and skip the generation
import importlib
importlib.reload(clw)
clt = clw.CodellamaEval(model_name = None, method_dict_location = 'output_multi-humaneval_CodeLlama-7b-hf.json')

### Refact

In [1]:
import causallm_wrapper as clmw

In [54]:
import importlib
importlib.reload(clmw)
refact = clmw.CausalLMEval(gpu_id=2)
refact.generate()

Loading multi-humaneval | language = kotlin


0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
1it [00:06,  6.80s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
2it [00:13,  6.90s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
3it [00:21,  7.16s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

In [55]:
file_name = refact.model_process()

In [56]:
refact.evaluate()

Reading samples...


161it [00:00, 11880.00it/s]


Running test suites...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161/161 [01:12<00:00,  2.22it/s]


Writing results to output_multi-humaneval_Refact-1_6B-fim.jsonl_results.jsonl...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161/161 [00:00<00:00, 49315.92it/s]


'pass rate: 19.88, test fail rate: 35.4, compilation error rate: 44.72, out of time rate: 0.0'

In [57]:
import jsonlines
with jsonlines.open(file_name) as reader:
    for obj in reader:
        print(obj['completion'])
        print('----')


    // Check if any two numbers are closer to each other than the threshold
    for (i in 0 until numbers.size - 1) {
        for (j in i + 1 until numbers.size) {
            if (Math.abs(numbers[i] - numbers[j]) < threshold) {
                return true
            }
        }
    }
    return false
}
----

    val result = mutableListOf<String>()
    var openBraces = 0
    var closeBraces = 0
    var currentString = ""

    for (char in parenString) {
        if (char == '(') {
            openBraces++
        } else if (char == ')') {
            closeBraces++
        }

        if (char == ')' && openBraces > 0) {
            openBraces--
        }

        if (openBraces == closeBraces) {
            currentString += char
        } else if (char =='') {
            continue
        } else {
            if (currentString.isNotEmpty()) {
                result.add(currentString)
                currentString = ""
            }
        }
    }

    if (currentString.isNotEmpty()) 

In [49]:
refact.method_dict['HumanEval_kotlin/24']

'/**\n * You are an expert Kotlin programmer, and here is your task.\n * For a given number n, find the largest number that divides n evenly, smaller than n\n * >>> largest_divisor(15)\n * 5\n *\n */\nfun largestDivisor(n : Int) : Int {\n    var largest = 0\n    for (i in 2..n) {\n        if (n % i == 0) {\n            largest = i\n        }\n    }\n    return largest\n}\n\n// Testing the function\nfun main() {\n    // Test case 1: n = 15\n    // The largest number that divides 15 evenly is 3, so the expected output is 5\n    println(largestDivisor(15)) // Output: 5\n\n    // Test case 2: n = 10\n    // The largest number that divides 10 evenly is 1, so the expected output is 1\n    println(largestDivisor(10)) // Output: 1\n\n    // Test case 3: n = 25\n    // The largest number that divides 25 evenly is 5, so the expected output is 5\n    println(largestDivisor(25)) // Output: 5\n\n    // Test case 4: n = 13\n    // The largest number that divides 13 evenly is 1, so the expected outpu

In [52]:
def my_filter(answer: str):
    answer_start = answer.find('{\n')
    answer_end = answer.find('\n}\n\n', answer_start + 1)
    return answer[answer_start+1:answer_end +2]

In [53]:
print(my_filter(refact.method_dict['HumanEval_kotlin/25']))


    val factors = mutableListOf<Int>()
    var num = n
    var i = 2
    while (i * i <= num) {
        while (num % i == 0) {
            factors.add(i)
            num /= i
        }
        i++
    }
    if (num > 1) {
        factors.add(num)
    }
    return factors
}


In [6]:
### Here I fiddled a bit with the model itself

prompt = "/**\n * You are an expert Kotlin programmer, and here is your task.\n * Insert a number 'delimeter' between every two consecutive elements of input list `numbers'\n * >>> intersperse([], 4)\n * []\n * >>> intersperse([1, 2, 3], 4)\n * [1, 4, 2, 4, 3]\n *\n */\nfun intersperse(numbers : List<Any>, delimeter : Int) : List<Any> {\n"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=200, temperature=0.2)
print("-"*80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


--------------------------------------------------------------------------------
/**
 * You are an expert Kotlin programmer, and here is your task.
 * Insert a number 'delimeter' between every two consecutive elements of input list `numbers'
 * >>> intersperse([], 4)
 * []
 * >>> intersperse([1, 2, 3], 4)
 * [1, 4, 2, 4, 3]
 *
 */
fun intersperse(numbers : List<Any>, delimeter : Int) : List<Any> {
    val result = mutableListOf<Any>()
    for (number in numbers) {
        result.add(number)
        if (result.size == 2) {
            result.add(delimeter)
        }
    }
    return result
}

/**
 * Test cases
 */
fun main() {
    // Test case 1
    val numbers1 = listOf(1, 2, 3)
   


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "smallcloudai/Refact-1_6B-fim"
device = "cuda:2" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

prompt = '<fim_prefix>def print_hello_world():\n    """<fim_suffix>\n    print("Hello world!")<fim_middle>'

inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=100, temperature=0.2)
print("-"*80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


--------------------------------------------------------------------------------
<fim_prefix>def print_hello_world():
    """<fim_suffix>
    print("Hello world!")<fim_middle>Prints 'Hello world!'"""<|endoftext|>


In [7]:
prompt_template = "<empty_output>SYSTEM {system}\n" \
                  "<empty_output>USER {query}\n" \
                  "<empty_output>ASSISTANT"
prompt = prompt_template.format(system="You are a programming assistant",
                                query="How do I sort a list in Python?")
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=100, temperature=0.2)
print("-"*80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


--------------------------------------------------------------------------------
<empty_output>SYSTEM You are a programming assistant
<empty_output>USER How do I sort a list in Python?
<empty_output>ASSISTANT To sort a list in Python, you can use the built-in sorted() function. For example, if you have a list called my_list and you want to sort it in ascending order, you can use the following code: sorted_list = sorted(my_list).
<empty_output><|endoftext|>
